In [1]:
from directory_cobveg_load import definir_diretorio 
project_path = definir_diretorio()

import geopandas as gpd
import os

from utils import(
    get_data_diretorio,
    check_crs,
    save_parquet_excel,
)
from utils.downloads import(
    download_malha_geosampa,
)

# Cobertura Vegetal 2020 (GeoSampa)

In [2]:
data_path = get_data_diretorio()

## Importar dados 

In [3]:
cache_path = os.path.join(
    data_path, 
    'cache'
)
cache_full_path = os.path.join(
        cache_path, 
        'cobertura_vegetal_concat.parquet'
                )
if os.path.exists(
    cache_full_path
):
    print('Carregando arquivo em cache')
    gdf_cobertura_vegetal = gpd.read_parquet(cache_full_path)
else:
    gdf_cobertura_vegetal = download_malha_geosampa(
        'cobertura_vegetal', 
        cache_path, 
        True
    )

Cuidado que talvez voce precise definir uma coluna de indice. A API retorna um documento com um erro e não dá status code correto!
http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=geoportal:cobertura_vegetal&maxFeatures=2000&startIndex=0
cobertura_vegetal_pg1
helloo
C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap\data\cache\cobertura_vegetal_pg1.zip
(2000, 19)
http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=geoportal:cobertura_vegetal&maxFeatures=2000&startIndex=2000
cobertura_vegetal_pg2
helloo
C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap\data\cache\cobertura_vegetal_pg2.zip
(2000, 19)
http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=geoportal:cobertura_vegetal&maxFeatures=2000&startIndex=4000
cobertura_vegetal_pg3
helloo
C:\Use

In [4]:
gdf_cobertura_vegetal.shape

(272117, 19)

In [5]:
gdf_cobertura_vegetal.sample(3)

,cd_identif,cd_identi0,cd_control,tx_datum,cd_coorden,cd_coorde0,cd_categor,cd_subcate,cd_subcat0,tx_descric,qt_area_ve,qt_perimet,qt_altura_,qt_desvio_,tx_data_vo,tx_entrega,tx_lote,tx_lote_ve,geometry
437,184438.0,184915.0,180697.0,"SIRGAS 2000,4 - 23S",331801.05,7380489.07,14.0,0.0,0.0,Vegetação herbáceo-arbustiva,110.05429,42.76270,0.0,0.0,2017,09_2018,009,V2,"POLYGON ((331793.298 7380489.514, 331798.771 7..."
1338,189339.0,189821.0,179252.0,"SIRGAS 2000,4 - 23S",330006.74,7380753.21,13.0,0.0,0.0,"Média a alta cobertura arbórea, arbóreo-arbust...",124.57488,44.10403,0.0,0.0,2017,09_2018,009,V2,"POLYGON ((330012.136 7380756.632, 330012.936 7..."
225,226.0,228.0,5292.0,"SIRGAS 2000,4 - 23S",329788.41,7397993.45,13.0,0.0,0.0,"Média a alta cobertura arbórea, arbóreo-arbust...",73.73369,34.38430,0.0,0.0,2017,04_2018,001,V2,"POLYGON ((329785.676 7397998.454, 329786.726 7..."


## Padronizaçã de colunas

Há uma série de colunas que podem ser importantes ou não. As que eu escolhi apagar são as que constam no dicionário 'drop_cobveg_cols' a seguir:

In [6]:
drop_cobveg_cols ={
    'cd_control',
    'tx_datum',
    'cd_coorden',
    'cd_coorde0',
    'tx_descric',
    'qt_area_ve',
    'qt_perimet',
    'qt_desvio_',
    'tx_data_vo',
    'tx_entrega',
    'tx_lote',
    'tx_lote_ve'    
}

gdf_cobertura_vegetal.drop(
    columns=drop_cobveg_cols,
    axis=1,
    inplace=True
)

In [7]:
rename_cobveg_cols ={
    'cd_identif' : 'cd_cob_veg'
}

gdf_cobertura_vegetal.rename(
    rename_cobveg_cols,
    axis=1,
    inplace=True
)

## Conferir CRS

In [8]:
gdf_cobertura_vegetal = check_crs(gdf_cobertura_vegetal)

## Salvar GDF

In [9]:
save_parquet_excel(
    gdf_cobertura_vegetal, 
    'malha_cobertura_vegetal',
    data_path,
    data_subpath='assets' 
)